# XGBOOST parameter exploration
This notebook is a template for finding the XGBOOST model best suited for your needs

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from xgboost import XGBClassifier
import sys
parent_dir=os.path.dirname(os.getcwd())
sys.path.insert(0,parent_dir )
import rippl_AI
import aux_fcn

### Data download
4 uLED sessions will be downloaded: Amigo2 and Som2 will be used for training ; Dlx1 and Thy7 for validation


In [ ]:
from figshare.figshare.figshare import Figshare
fshare = Figshare()

article_ids = [16847521,16856137,14959449,14960085] 
sess=['Amigo2','Som2','Dlx1','Thy7']                                  
for id,s in zip(article_ids,sess):
    datapath = os.path.join(parent_dir,'Downloaded_data', f'{s}')
    if os.path.isdir(datapath):
        print(f"{s} session already exists. Moving on.")
    else:
        print("Downloading data... Please wait, this might take up some time")        # Can take up to 10 minutes
        fshare.retrieve_files_from_article(id,directory=datapath)
        print("Data downloaded!")

### Data load
The training sessions' LFP will be appended together in a list. The same will happen with the ripples detection times.
That is the required input for the training parser

In [ ]:
# The training sessions will be appended together. Replace this cell with your own data loading
train_LFPs=[]
train_GTs=[]
# Amigo2
path=os.path.join(parent_dir,'Downloaded_data','Amigo2','figshare_16847521')
LFP,GT=aux_fcn.load_lab_data(path)
train_LFPs.append(LFP)
train_GTs.append(GT)
# Som2
path=os.path.join(parent_dir,'Downloaded_data','Som2','figshare_16856137')
LFP,GT=aux_fcn.load_lab_data(path)
train_LFPs.append(LFP)
train_GTs.append(GT)

## Append all your validation sessions
val_LFPs=[]
val_GTs=[]
# Dlx1 Validation
path=os.path.join(parent_dir,'Downloaded_data','Dlx1','figshare_14959449')
LFP,GT=aux_fcn.load_lab_data(path)
val_LFPs.append(LFP)
val_GTs.append(GT)
# Thy07 Validation
path=os.path.join(parent_dir,'Downloaded_data','Thy7','figshare_14960085')
LFP,GT=aux_fcn.load_lab_data(path)
val_LFPs.append(LFP)
val_GTs.append(GT)

x_training,GT_training,x_val_list,GT_val_list=rippl_AI.prepare_training_data(train_LFPs,train_GTs,val_LFPs,val_GTs,sf=30000)

## XGBOOST training parameters

#### Parameters:
* Channels:  number of channels that will be used to train the model, extracted from the data shape defined in the previous cell
* Timesteps: number of samples that the will be used to generate a single prediction
* Max depth: number of max layers in each tree. Too many usually causes overfitting
* Learning rate: similar to a weight used to update te predictor, a high value leads to faster computations but may not reaach a optimal value
* Gamma: Minimum loss reduction required to make a partition on a leaf node. The larger gamma is, the more conservative the model will be
* Reg lamda: L2 regularization term of weight updating. Increasing this value makes the model more conservative
* Scale pos weight: controls the balance of positive and negative weights, useful for unbalanced clasess.
* Subsample: subsample ratio of the training instances. Setting it to 0.5 means that XGBoost would randomly sample half of the training data prior to growing trees. Used to prevent overfitting
#
XGBOOST contains many more parameters, feel free to add your own modifications. Check the oficial documentation: https://xgboost.readthedocs.io/en/stable/parameter.html#parameters-for-tree-booster

In [ ]:
conf=   {"timesteps":[16],          # 2,4,8,16,20,32 ...
        "max_depth": [4, 5],          # 3,4,5,6,7 ...
        "learning_rate": [0.1], # 0.2, 0.1, 0.05, 0.01 ...
        "gamma": [1],                 # 0, 0.25, 1 ...
        "reg_lambda": [10],           # 0, 1, 10 ...
        "scale_pos_weight": [1],      # 1, 3, 5...
        "subsample": [0.8]}           # 0.5, 0.8, 0.9 ...

### Training

In [ ]:
# Desired sampling frequency of the models
sf=1250
th_arr=np.linspace(0.1,0.9,9)
model_name_arr=[]           # To plot in the next cell
model_arr=[]                # Actual model array, used in the next validation section
n_channels=x_training.shape[1]
timesteps_arr=conf['timesteps']

max_depth_arr=conf["max_depth"]        
lr_arr=conf["learning_rate"]
gamma_arr=conf["gamma"]                                             
reg_lambda_arr=conf["reg_lambda"]         
scale_arr=conf["scale_pos_weight"]
subsample_arr=conf["subsample"]      

l_ts=len(timesteps_arr)

l_maxd=len(max_depth_arr)
l_lr =len(lr_arr)
l_g =len(gamma_arr)
l_reg =len(reg_lambda_arr)
l_sc =len(scale_arr)
l_sub =len(subsample_arr)
n_iters=l_ts*l_maxd*l_lr*l_g*l_reg*l_sc*l_sub
# GT is in the shape (n_events x 2), a y output signal with the same length as x is required
perf_train_arr=np.zeros(shape=(n_iters,len(th_arr),3)) # Performance array, (n_models x n_th x 3 ) [P R F1]
perf_test_arr=np.zeros_like(perf_train_arr)
timesteps_arr_ploting=[]            # Array that will be used in the validation, to be able to call the function predict

print(f'{n_channels} channels will be used to train the XGBOOST models')

print(f'{n_iters} models will be trained')

x_test_or,GT_test,x_train_or,GT_train=aux_fcn.split_data(x_training,GT_training,split=0.7,sf=sf)

y_test_or= np.zeros(shape=(len(x_test_or)))
for ev in GT_test:
    y_test_or[int(sf*ev[0]):int(sf*ev[1])]=1
y_train_or= np.zeros(shape=(len(x_train_or)))
for ev in GT_train:
    y_train_or[int(sf*ev[0]):int(sf*ev[1])]=1


for i_ts,timesteps in enumerate(timesteps_arr):

        x_train=x_train_or[:len(x_train_or)-len(x_train_or)%timesteps].reshape(-1,timesteps*n_channels)
        y_train_aux=y_train_or[:len(y_train_or)-len(y_train_or)%timesteps].reshape(-1,timesteps)
        y_train=aux_fcn.rec_signal(y_train_aux) # If any sample of the window contains a ripple, the desired output for the shape is 1
        
        x_test=x_test_or[:len(x_test_or)-len(x_test_or)%timesteps].reshape(-1,timesteps*n_channels)
        y_test_aux=y_test_or[:len(y_test_or)-len(y_test_or)%timesteps].reshape(-1,timesteps)
        y_test=aux_fcn.rec_signal(y_test_aux)

        for i_maxd,max_depth in enumerate(max_depth_arr):
                for i_lr,lr in enumerate(lr_arr):
                    for i_g,g in enumerate(gamma_arr):
                        for i_rg,reg_l in enumerate(reg_lambda_arr):
                            for i_sc,scale in enumerate(scale_arr):
                                for i_subs,subsample in enumerate(subsample_arr):
                                    iter=(((((i_ts*l_maxd+i_maxd)*l_lr+i_lr)*l_g+i_g)*l_reg+ i_rg)*l_sc + i_sc)*l_sub + i_subs
                                    print(f"\nIteration {iter+1} out of {n_iters}")
                                    print(f'Number of channels: {n_channels:d}, Time steps: {timesteps:d}.\nMax depth: {max_depth:d}, Lr: {lr:1.3f}, gamma: {g:1.2f}, reg_l: {reg_l:d}, scale: {scale:1.3f}, subsample: {subsample:0.3f}')
                                    xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                                    colsample_bynode=1, colsample_bytree=1, gamma=g, gpu_id=-1,
                                    importance_type='gain', interaction_constraints='',
                                    learning_rate=lr, max_delta_step=0, max_depth=max_depth,
                                    min_child_weight=1, monotone_constraints='()',
                                    n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=0,
                                    reg_alpha=0, reg_lambda=reg_l, scale_pos_weight=scale, subsample=subsample,
                                    tree_method='exact', validate_parameters=1, verbosity=2)

                                    # Training
                                    xgb.fit(x_train, y_train,verbose=1,eval_metric=["logloss"] ,eval_set = [(x_train,y_train),(x_test, y_test)])
                                    model_arr.append(xgb)
                                    # Prediction. One value for window
                                    test_signal = xgb.predict_proba(x_test)[:,1]
                                    train_signal=xgb.predict_proba(x_train)[:,1]
                                    # Not compatible with the functions that extract beginning and end times
                                    y_train_predict=np.empty(shape=(x_train.shape[0]*timesteps,1,1))
                                    for i,window in enumerate(train_signal):
                                        y_train_predict[i*timesteps:(i+1)*timesteps]=window
                                    
                                    y_test_predict=np.empty(shape=(x_test.shape[0]*timesteps,1,1))
                                    for i,window in enumerate(test_signal):
                                        y_test_predict[i*timesteps:(i+1)*timesteps]=window
        
                                    for i,th in enumerate(th_arr):
                                        # Test
                                        ytest_pred_ind=aux_fcn.get_predictions_index(y_test_predict,th)/sf
                                        perf_test_arr[iter,i]=aux_fcn.get_performance(ytest_pred_ind,GT_test,0)[0:3]
                                        # Train
                                        ytrain_pred_ind=aux_fcn.get_predictions_index(y_train_predict,th)/sf
                                        perf_train_arr[iter,i]=aux_fcn.get_performance(ytrain_pred_ind,GT_train,0)[0:3]

                                    # Saving the model
                                    model_name=f"XGBOOST_Ch{n_channels:d}_Ts{timesteps:03d}_D{max_depth:d}_Lr{lr:1.3f}_G{g:1.2f}_regl{reg_l:02d}_SCALE{scale:03d}_Subs{subsample:1.3f}"
                                    xgb.save_model(os.path.join(parent_dir,'explore_models',model_name))

                                    model_name_arr.append(model_name)
                                    timesteps_arr_ploting.append(timesteps)

### Plot training results

In [ ]:
# Plot training results
fig,axs=plt.subplots(n_iters,2,figsize=(10,2*n_iters),sharey='col',sharex='col')

for i in range(n_iters):
    axs[i,0].plot(perf_train_arr[i,:,0],perf_train_arr[i,:,1],'k.-')
    axs[i,0].plot(perf_test_arr[i,:,0],perf_test_arr[i,:,1],'b.-')
    axs[i,1].plot(th_arr,perf_train_arr[i,:,2],'k.-')
    axs[i,1].plot(th_arr,perf_test_arr[i,:,2],'b.-')
    axs[i,0].set_title(model_name_arr[i])
    axs[i,0].set_ylabel('Precision')
    axs[i,1].set_ylabel('F1')
axs[-1,0].set_xlabel('Recall')
axs[-1,1].set_xlabel('Threshold')
axs[0,0].legend(['Training','Test'])
plt.show()

### Validation

In [ ]:
# For loop iterating over the models
fig,axs=plt.subplots(n_iters,2,figsize=(10,2*n_iters),sharey='col',sharex='col')
for n_m,model in enumerate(model_arr):
    F1_arr=np.zeros(shape=(len(x_val_list),len(th_arr))) #(n_val_sess x n_th) Array where the F1 val of each sesion will be stored
    for n_sess,LFP in enumerate(x_val_list):
        val_pred=rippl_AI.predict(LFP,sf=1250,arch='XGBOOST',new_model=model,n_channels=n_channels,n_timesteps=timesteps_arr_ploting[n_m])[0]
        for i,th in enumerate(th_arr):
            val_pred_ind=aux_fcn.get_predictions_index(val_pred,th)/sf
            F1_arr[n_sess,i]=aux_fcn.get_performance(val_pred_ind,GT_val_list[n_sess],verbose=False)[2]
    
    axs[n_m,0].plot(th_arr,perf_train_arr[n_m,:,2],'k.-')
    axs[n_m,0].plot(th_arr,perf_test_arr[n_m,:,2],'b.-')
    for F1 in F1_arr:
        axs[n_m,1].plot(th_arr,F1)
    axs[n_m,1].plot(th_arr,np.mean(F1_arr,axis=0),'k.-')
    axs[n_m,0].set_title(model_name_arr[n_m])
    axs[n_m,0].set_ylabel('Precision')
    axs[n_m,1].set_ylabel('F1')
axs[-1,0].set_xlabel('Recall')
axs[-1,1].set_xlabel('Threshold')
plt.show()
    